<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `Greedy` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [7]:
# External imports
from typing import *
from typing_extensions import *
from numbers import *
import unittest
import sys
import os
import random

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from Greedy import Greedy
from pyrat import BigHolesRandomMaze, Action,MazeFromDict

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [8]:
class GreedyTests (unittest.TestCase):

    """
        This class tests the methods of the Greedy class.
        For each method, we test it with a few different configurations.
    """

    #############################################################################################################################################
    #                                                                 UNIT TESTS                                                                #
    #############################################################################################################################################

    def test_traversal_random ( self: Self
                              ) ->    None:

        """
            This method tests the 'traversal' method.
            We are going to check the following:
            - Outputs are of the expected types.
            - All vertices are visited.
            - The routing table is a tree, with root corresponding to the start of the traversal.
            - The found distances are correct, i.e., strictly positive and increasing, except for the start vertex which should be 0.
            Note that we cannot test that the output is unique, as there are multiple valid outputs, depending on how vertices are explored.
            We will test the method on several random graphs to make sure it is working properly.
            Random graphs will be generated using the PyRat class used to create mazes in games.
            There are several such classes, but we will use the BigHolesRandomMaze class.
            To test that we indeed find the shortest path, we also test the method on some controlled examples in a separate test function.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        NB_GRAPHS = 10
        WIDTHS = [2, 30]
        HEIGHTS = [2, 30]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 10.0

        # Test on several graphs
        for i in range(NB_GRAPHS):
            
            # Instantiate the player
            player = Greedy()

            # Generate a random maze
            # We use a fixed random seed for reproducibility of tests
            rng = random.Random(i)
            maze = BigHolesRandomMaze(width = rng.randint(WIDTHS[0], WIDTHS[1]),
                                      height = rng.randint(HEIGHTS[0], HEIGHTS[1]),
                                      cell_percentage = rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                                      wall_percentage = rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                                      mud_percentage = MUD_PERCENTAGE,
                                      mud_range=(4, 9),
                                      random_seed = i)
            # Choose a number of pieces of cheese then randomly select their position in the maze
            nb_cheese=random.randint(1, len(maze.vertices)//2)
            pieces_of_cheese=random.sample(maze.vertices, nb_cheese)

            # Choose a random starting vertex that isn't a peice of cheese
            start_vertex = random.choice([v for v in maze.vertices if v not in pieces_of_cheese])

            # Perform the traversal
            distances, routing_table = player.traversal(maze, start_vertex,pieces_of_cheese)

            # Check the output type for distances
            # It should be a dictionary with integer keys and integer values
            self.assertTrue(isinstance(distances, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in distances.keys()))
            self.assertTrue(all(isinstance(v, Integral) for v in distances.values()))
            
            # Check the output type for the routing table
            # It should be a dictionary with integer keys and integer or None values
            self.assertTrue(isinstance(routing_table, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in routing_table.keys()))
            self.assertTrue(all(isinstance(v, (type(None), Integral)) for v in routing_table.values()))

            # All pieces of cheese should be visited
            # Equivalently, the distances should have the same keys as the pieces of cheese
            self.assertTrue(all(peice in set(distances.keys()) for peice in pieces_of_cheese))

            # Check that the start vertex is the only tree root
            self.assertEqual(routing_table[start_vertex], None)
            self.assertTrue(all(routing_table[v] is not None for v in routing_table.keys() if v != start_vertex))
            self.assertEqual(distances[start_vertex], 0)
            self.assertTrue(all(distances[v] > 0 for v in distances.keys() if v != start_vertex))
            
            # We check that the routing table is a tree
            # Also we check that associated distances are decreasing as we go to the root
            for peice in pieces_of_cheese:

                # We check that the parent is closer to the root
                if routing_table[peice] is not None:
                    self.assertTrue(distances[routing_table[peice]] < distances[peice])
                
                # We ckeck that we can reach the root from any peice
                path = [peice]
                while routing_table[path[-1]] is not None:
                    self.assertTrue(routing_table[path[-1]] not in path)
                    path.append(routing_table[path[-1]])
    
    #############################################################################################################################################

    def test_traversal_fixed ( self: Self
                             ) ->    None:

        """
            This method complements the previous tests by testing the 'traversal' method on some controlled examples.
            To do so, we will use some mazes for which there is a unique routing table.
            We do not test the output types again, as it has already been done in the previous test function.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        TESTS = [{"inputs": {"maze": MazeFromDict({0: {6: 1}, 1: {7: 1}, 2: {8: 1}, 3: {4: 1}, 4: {3: 1, 5: 1, 10: 1}, 5: {4: 1}, 6: {0: 1, 12: 1, 7: 1}, 
                                                7: {1: 1, 8: 1, 6: 1, 13: 5}, 8: {2: 1, 7: 1, 14: 8}, 10: {4: 1, 11: 1, 16: 8}, 11: {10: 1, 17: 1}, 
                                                12: {6: 1, 13: 1}, 13: {12: 1, 14: 1, 19: 1, 7: 5}, 14: {13: 1, 8: 8}, 16: {22: 1, 10: 8, 17: 4}, 
                                                17: {23: 1, 11: 1, 16: 4}, 19: {13: 1, 25: 5}, 22: {16: 1, 23: 1, 28: 1}, 23: {22: 1, 17: 1}, 25: {26: 1, 19: 5}, 
                                                26: {25: 1, 27: 1}, 27: {26: 1, 28: 1}, 28: {22: 1, 27: 1, 29: 1}, 29: {28: 1}})    ,
                             "start": 13,
                             "pieces_of_cheese":[5,10,22]},
                  "outputs": {"routing_table": {13: None, 7: 6, 12: 13, 14: 13, 19: 13, 6: 12, 8: 7, 25: 19, 0: 6, 1: 7, 2: 8, 26: 25, 27: 26, 28: 27, 22: 28, 29: 28, 16: 22, 23: 22, 10: 11, 17: 23, 11: 17, 4: 10, 3: 4, 5: 4},
                              "distances": {13: 0, 7: 3, 12: 1, 14: 1, 19: 1, 6: 2, 8: 4, 25: 6, 0: 3, 1: 4, 2: 5, 26: 7, 27: 8, 28: 9, 22: 10, 29: 10, 16: 11, 23: 11, 10: 14, 17: 12, 11: 13, 4: 15, 3: 16, 5: 16}}},
                 {"inputs": {"maze": MazeFromDict({0: {1: 1, 6: 1}, 1: {0: 1, 2: 1, 7: 1}, 2: {1: 1, 3: 1}, 3: {2: 1, 4: 1}, 4: {3: 1, 5: 1}, 5: {4: 1}, 6: {0: 1, 7: 1, 12: 1}, 7: {6: 1, 13: 1, 1: 1, 8: 7}, 8: {9: 1, 7: 7}, 9: {8: 1, 15: 1}, 12: {6: 1, 13: 1, 18: 8}, 13: {12: 1, 14: 1, 19: 1, 7: 1}, 14: {13: 1, 15: 1}, 15: {14: 1, 21: 1, 9: 1}, 18: {12: 8, 24: 6}, 19: {20: 1, 25: 1, 13: 1}, 20: {19: 1, 21: 5}, 21: {15: 1, 27: 1, 20: 5}, 24: {25: 5, 18: 6}, 25: {19: 1, 24: 5}, 26: {27: 1}, 27: {21: 1, 26: 1, 28: 1}, 28: {27: 1, 29: 1}, 29: {28: 1}}),
                             "start":6,
                             "pieces_of_cheese":[8,16,21]},
                  "outputs": {"routing_table": {6: None, 0: 6, 7: 6, 12: 6, 1: 0, 13: 7, 8: 9, 18: 12, 2: 1, 14: 13, 19: 13, 3: 2, 15: 14, 20: 19, 25: 19, 4: 3, 21: 15, 9: 15, 24: 25, 5: 4, 27: 21, 26: 27, 28: 27, 29: 28},
                              "distances": {6: 0, 0: 1, 7: 1, 12: 1, 1: 2, 13: 2, 8: 6, 18: 9, 2: 3, 14: 3, 19: 3, 3: 4, 15: 4, 20: 4, 25: 4, 4: 5, 21: 5, 9: 5, 24: 9, 5: 6, 27: 6, 26: 7, 28: 7, 29: 8}}}]
                
        # Test on several controlled examples
        for i in range(len(TESTS)):
            
            # Instantiate the player
            player = Greedy()

            # Get the input and output data
            maze = TESTS[i]["inputs"]["maze"]
            start = TESTS[i]["inputs"]["start"]
            targets=TESTS[i]["inputs"]["pieces_of_cheese"]
            target_routing_table = TESTS[i]["outputs"]["routing_table"]
            target_distances = TESTS[i]["outputs"]["distances"]

            # Perform the traversal
            distances, routing_table = player.traversal(maze, start,targets)

            # Check that outputs match the expected ones
            self.assertEqual(sorted(routing_table), sorted(target_routing_table))
            self.assertEqual(sorted(distances), sorted(target_distances))

    #############################################################################################################################################

    def test_find_route_and_distance ( self: Self,
                        ) ->    None:
        
        """
            This method tests the 'find_route_and_distance' method.
            Here, we want to make sure that the output corresponds indeed to a route from the start to the end.
            We are going to check the following:
            - Outputs are of the expected types.
            - The route is a valid path from the start to the end.
            The method will be tested on some controlled examples, where we can easily check the validity of the output.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        INPUTS = [{"routing_table": {0: None, 1: 0, 2: 1, 3: 2, 4: 3},
                   "distances":{0: 0, 1: 1, 2: 2, 3: 3, 4: 4},
                       "start": 0,
                       "end": 4},
                  {"routing_table": {0: 2, 1: None, 2: 1, 3: 0, 4: 0, 5: 4, 6: 1}, 
                   "distances":{1: 0 , 0: 2, 6: 1, 2: 1, 4: 3, 5: 4, 3: 3},
                       "start": 1,
                       "end": 6}]

        # Test on several controlled examples
        for i in range(len(INPUTS)):
            
            # Instantiate the player
            player = Greedy()

            # Get the input data
            routing_table = INPUTS[i]["routing_table"]
            distances = INPUTS[i]["distances"]
            start = INPUTS[i]["start"]
            end = INPUTS[i]["end"]

            # Find the route
            route,distance = player.find_route_and_distance(routing_table,distances, start, end)

            # Check the output type
            # It should be a list of integers
            self.assertTrue(isinstance(route, List))
            self.assertTrue(all(isinstance(v, Integral) for v in route))

            # Check that the route is a valid path from the start to the end
            self.assertEqual(route[0], start)
            self.assertEqual(route[-1], end)
            self.assertTrue(all(routing_table[route[j + 1]] == route[j] for j in range(len(route) - 1)))

            # Check that the distance returned is true
            self.assertEqual(distance,distances[end])
    
    #############################################################################################################################################
    def test_meta_graph(self):
        """
            This method tests the 'meta_graph' method.
            Here, we want to make sure that the output of the method is as we expect it to be.
            The method will be tested on some controlled examples, where we can easily check the validity of the output.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """
        # Define the maze
        maze =MazeFromDict({0: {6: 1}, 1: {7: 1}, 2: {8: 1}, 3: {4: 1}, 4: {3: 1, 5: 1, 10: 1}, 5: {4: 1}, 6: {0: 1, 12: 1, 7: 1}, 
                            7: {1: 1, 8: 1, 6: 1, 13: 5}, 8: {2: 1, 7: 1, 14: 8}, 10: {4: 1, 11: 1, 16: 8}, 11: {10: 1, 17: 1}, 
                            12: {6: 1, 13: 1}, 13: {12: 1, 14: 1, 19: 1, 7: 5}, 14: {13: 1, 8: 8}, 16: {22: 1, 10: 8, 17: 4}, 
                            17: {23: 1, 11: 1, 16: 4}, 19: {13: 1, 25: 5}, 22: {16: 1, 23: 1, 28: 1}, 23: {22: 1, 17: 1}, 25: {26: 1, 19: 5}, 
                            26: {25: 1, 27: 1}, 27: {26: 1, 28: 1}, 28: {22: 1, 27: 1, 29: 1}, 29: {28: 1}})
        # Instantiate the player
        player = Greedy()
        source = 0
        cheese = [5,10,22]
        OUTPUT={5: {0: (19, [5, 4, 10, 11, 17, 23, 22, 28, 27, 26, 25, 19, 13, 12, 6, 0]), 22: (6, [5, 4, 10, 11, 17, 23, 22]), 10: (2, [5, 4, 10])}, 
                10: {0: (17, [10, 11, 17, 23, 22, 28, 27, 26, 25, 19, 13, 12, 6, 0]), 22: (4, [10, 11, 17, 23, 22]), 5: (2, [10, 4, 5])}, 
                22: {0: (13, [22, 28, 27, 26, 25, 19, 13, 12, 6, 0]), 5: (6, [22, 23, 17, 11, 10, 4, 5]), 10: (4, [22, 23, 17, 11, 10])}, 
                0: {5: (19, [0, 6, 12, 13, 19, 25, 26, 27, 28, 22, 23, 17, 11, 10, 4, 5]), 10: (17, [0, 6, 12, 13, 19, 25, 26, 27, 28, 22, 23, 17, 11, 10]), 22: (13, [0, 6, 12, 13, 19, 25, 26, 27, 28, 22])}}
        # Check that the output is true
        self.assertEqual(player.meta_graph(maze,source,cheese),OUTPUT)

    #############################################################################################################################################

    def test_partial_path(self:Self)  ->    None:
        """
            This method tests the 'partial_path' method.
            Here, we want to make sure that the output corresponds indeed the correct one 
            We are going to check the following:
            - Outputs are of the expected types.
            - The partial path given by the method is the correct one .
            The method will be tested on some controlled examples, where we can easily check the validity of the output.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """        
        INPUTS = [{"meta_graph": {11: {7: (1, [11, 7]), 0: (19, [11, 7, 6, 5, 4, 0]), 2: (3, [11, 7, 3, 2]), 6: (2, [11, 7, 6])}, 6: {7: (1, [6, 7]), 0: (17, [6, 5, 4, 0]), 2: (1, [6, 2]), 11: (2, [6, 7, 11])}, 2: {7: (2, [2, 3, 7]), 0: (18, [2, 6, 5, 4, 0]), 11: (3, [2, 3, 7, 11]), 6: (1, [2, 6])}, 0: {7: (18, [0, 4, 5, 6, 7]), 11: (19, [0, 4, 5, 6, 7, 11]), 6: (17, [0, 4, 5, 6]), 2: (18, [0, 4, 5, 6, 2])}, 7: {11: (1, [7, 11]), 6: (1, [7, 6]), 2: (2, [7, 3, 2]), 0: (18, [7, 6, 5, 4, 0])}},
                   "peices_of_cheese":[11,6,2,0],
                   "start": 7},
                  {"meta_graph":{22: {12: (7, [22, 21, 20, 14, 13, 7, 6, 12]), 1: (6, [22, 21, 20, 14, 13, 7, 1]), 18: (6, [22, 21, 20, 14, 13, 19, 18]), 2: (7, [22, 21, 20, 14, 13, 7, 1, 2]), 13: (4, [22, 21, 20, 14, 13])}, 13: {12: (3, [13, 7, 6, 12]), 1: (2, [13, 7, 1]), 18: (2, [13, 19, 18]), 2: (3, [13, 7, 1, 2]), 22: (4, [13, 14, 20, 21, 22])}, 2: {12: (4, [2, 1, 7, 6, 12]), 1: (1, [2, 1]), 18: (5, [2, 1, 7, 13, 19, 18]), 22: (7, [2, 1, 7, 13, 14, 20, 21, 22]), 13: (3, [2, 1, 7, 13])}, 18: {12: (5, [18, 19, 13, 7, 6, 12]), 1: (4, [18, 19, 13, 7, 1]), 22: (6, [18, 19, 13, 14, 20, 21, 22]), 13: (2, [18, 19, 13]), 2: (5, [18, 19, 13, 7, 1, 2])}, 1: {12: (3, [1, 7, 6, 12]), 22: (6, [1, 7, 13, 14, 20, 21, 22]), 13: (2, [1, 7, 13]), 2: (1, [1, 2]), 18: (4, [1, 7, 13, 19, 18])}, 12: {22: (7, [12, 6, 7, 13, 14, 20, 21, 22]), 13: (3, [12, 6, 7, 13]), 2: (4, [12, 6, 7, 1, 2]), 18: (5, [12, 6, 7, 13, 19, 18]), 1: (3, [12, 6, 7, 1])}},
                   "peices_of_cheese":[22,13,2,18,1],
                   "start":12 }]
        OUTPUTS=[[7, 11, 6, 2, 0],[12, 13, 18, 1, 2, 22]]
        # Test on several controlled examples
        for i in range(len(INPUTS)):
            
            # Instantiate the player
            player = Greedy()
            output=OUTPUTS[i]

            # Get the input data
            meta_graph = INPUTS[i]["meta_graph"]
            peices_of_cheese= INPUTS[i]["peices_of_cheese"]
            start = INPUTS[i]["start"]

            # Find the route
            partial_path = player.partial_path(peices_of_cheese,start,meta_graph)

            # Check the output type
            # It should be a list of integers
            self.assertTrue(isinstance(partial_path, List))
            self.assertTrue(all(isinstance(v, Integral) for v in partial_path))

            # Check that the route is a valid path from the start to the end
            self.assertTrue(partial_path,output)            
            

    def test_complete_path ( self: Self,
                        ) ->    None:
        
        """
            This method tests the 'complete_path' method.
            Here, we want to make sure that the output corresponds indeed the correct one 
            We are going to check the following:
            - Outputs are of the expected types.
            - The route is the correct path from the start to the end.
            The method will be tested on some controlled examples, where we can easily check the validity of the output.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        INPUTS = [{"meta_graph": {11: {7: (1, [11, 7]), 0: (19, [11, 7, 6, 5, 4, 0]), 2: (3, [11, 7, 3, 2]), 6: (2, [11, 7, 6])}, 6: {7: (1, [6, 7]), 0: (17, [6, 5, 4, 0]), 2: (1, [6, 2]), 11: (2, [6, 7, 11])}, 2: {7: (2, [2, 3, 7]), 0: (18, [2, 6, 5, 4, 0]), 11: (3, [2, 3, 7, 11]), 6: (1, [2, 6])}, 0: {7: (18, [0, 4, 5, 6, 7]), 11: (19, [0, 4, 5, 6, 7, 11]), 6: (17, [0, 4, 5, 6]), 2: (18, [0, 4, 5, 6, 2])}, 7: {11: (1, [7, 11]), 6: (1, [7, 6]), 2: (2, [7, 3, 2]), 0: (18, [7, 6, 5, 4, 0])}},
                       "partial_path":[7, 11, 6, 2, 0],
                       "end": 0},
                  {"meta_graph":{22: {12: (7, [22, 21, 20, 14, 13, 7, 6, 12]), 1: (6, [22, 21, 20, 14, 13, 7, 1]), 18: (6, [22, 21, 20, 14, 13, 19, 18]), 2: (7, [22, 21, 20, 14, 13, 7, 1, 2]), 13: (4, [22, 21, 20, 14, 13])}, 13: {12: (3, [13, 7, 6, 12]), 1: (2, [13, 7, 1]), 18: (2, [13, 19, 18]), 2: (3, [13, 7, 1, 2]), 22: (4, [13, 14, 20, 21, 22])}, 2: {12: (4, [2, 1, 7, 6, 12]), 1: (1, [2, 1]), 18: (5, [2, 1, 7, 13, 19, 18]), 22: (7, [2, 1, 7, 13, 14, 20, 21, 22]), 13: (3, [2, 1, 7, 13])}, 18: {12: (5, [18, 19, 13, 7, 6, 12]), 1: (4, [18, 19, 13, 7, 1]), 22: (6, [18, 19, 13, 14, 20, 21, 22]), 13: (2, [18, 19, 13]), 2: (5, [18, 19, 13, 7, 1, 2])}, 1: {12: (3, [1, 7, 6, 12]), 22: (6, [1, 7, 13, 14, 20, 21, 22]), 13: (2, [1, 7, 13]), 2: (1, [1, 2]), 18: (4, [1, 7, 13, 19, 18])}, 12: {22: (7, [12, 6, 7, 13, 14, 20, 21, 22]), 13: (3, [12, 6, 7, 13]), 2: (4, [12, 6, 7, 1, 2]), 18: (5, [12, 6, 7, 13, 19, 18]), 1: (3, [12, 6, 7, 1])}},
                       "partial_path":[12, 13, 18, 1, 2, 22],
                       "end":22 }]
        OUTPUTS=[[7, 11, 7, 6, 2, 6, 5, 4, 0],[12, 6, 7, 13, 19, 18, 19, 13, 7, 1, 2, 1, 7, 13, 14, 20, 21, 22]]

        # Test on several controlled examples
        for i in range(len(INPUTS)):
            
            # Instantiate the player
            player = Greedy()
            output=OUTPUTS[i]

            # Get the input data
            meta_graph = INPUTS[i]["meta_graph"]
            partial_path= INPUTS[i]["partial_path"]
            end = INPUTS[i]["end"]

            # Find the route
            route = player.complete_path(partial_path,meta_graph)

            # Check the output type
            # It should be a list of integers
            self.assertTrue(isinstance(route, List))
            self.assertTrue(all(isinstance(v, Integral) for v in route))

            # Check that the route is a valid path from the start to the end
            self.assertEqual(route[-1], end)
            self.assertTrue(route,output)            


<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [9]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False)

test_complete_path (__main__.GreedyTests.test_complete_path)
This method tests the 'complete_path' method. ... ok
test_find_route_and_distance (__main__.GreedyTests.test_find_route_and_distance)
This method tests the 'find_route_and_distance' method. ... ok
test_meta_graph (__main__.GreedyTests.test_meta_graph)
This method tests the 'meta_graph' method. ... ok
test_partial_path (__main__.GreedyTests.test_partial_path)
This method tests the 'partial_path' method. ... ok
test_traversal_fixed (__main__.GreedyTests.test_traversal_fixed)
This method complements the previous tests by testing the 'traversal' method on some controlled examples. ... ok
test_traversal_random (__main__.GreedyTests.test_traversal_random)
This method tests the 'traversal' method. ... ok

----------------------------------------------------------------------
Ran 6 tests in 5.285s

OK
